# ทำนายรายได้ประชากรด้วย GaussianNB จากข้อมูลที่เก็บจริง ไม่ได้โหลดมากจาก sklearn

ดู Attribute เช่น การศึกษา เพศ การแต่งงาน จากนั้นประเมินหา income ว่าจะได้มากกว่าหรือน้อยกว่า 50k<br>
Attribute คือ x<br>
มากกว่าหรือน้อยกว่า 50k เป็น Class คือ y_Test ผลเฉลย<br>


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# func Clean Data เพราะมีข้อมูลบางส่วนที่เป็นไม่ระบุค่า หรือตัวเลขแปลกๆ
# ในที่นี้เราจะจัดทำเข้ารหัสข้อความให้เป็นตัวเลขทั้งหมด
def cleanData(dataset):
    for column in dataset.columns:
        if dataset[column].dtype == type(object): #วนหาค่า dataset ในแต่ละ idx ว่าเป็น type object หรือไม่
            lbEncoder = LabelEncoder()
            dataset[column] = lbEncoder.fit_transform(dataset[column]) #เข้ารหัสข้อความให้เป็นตัวเลข
    return dataset

In [4]:
# func split แบ่งเพื่อเลือก Attribute(เลือก age -> country) และ class(เลือก income) แยกเก็บลงในตัวแปร
def split_Attribute_Class(dataset,columnName): 
    attribute_Select = dataset.drop(columnName, axis=1) #Attribute(เลือก age -> country)
    class_Select = dataset[columnName].copy() #class(เลือก income)
    return attribute_Select, class_Select

In [6]:
#local
path = "https://raw.githubusercontent.com/PNYWT/Python_MachineLearning/main/Mechine%20Learning/FileData/adult_census_income.csv"
dataset = pd.read_csv(path)
#หลังจาก Clean Data แล้ว income <= 50k คือ 0 และ >=50k คือ 1
dataset = cleanData(dataset)
print(dataset.head())

   age  workclass  fnlwgt  education  education.num  marital.status  \
0   90          0   77053         11              9               6   
1   82          4  132870         11              9               6   
2   66          0  186061         15             10               6   
3   54          4  140359          5              4               0   
4   41          4  264663         15             10               5   

   occupation  relationship  race  sex  capital.gain  capital.loss  \
0           0             1     4    0             0          4356   
1           4             1     4    0             0          4356   
2           0             4     2    0             0          4356   
3           7             4     4    0             0          3900   
4          10             3     4    0             0          3900   

   hours.per.week  native.country  income  
0              40              39       0  
1              18              39       0  
2              40   

In [7]:
# Split data จาก Data ทั้งหมดแยกเป็น Training-Test
data_Training, data_Test = train_test_split(dataset, train_size=0.7, test_size=0.3)

#เลือก Attribute(เลือก age -> country) และเลือก class(เลือก income) เก็บในตัวแปร สำหรับเข้า Traning model
x_Training, y_Training = split_Attribute_Class(data_Training, "income")
#เลือก Attribute(เลือก age -> country) และเลือก class(เลือก income) เก็บในตัวแปร สำหรับเข้า Test model
x_Test, y_Test = split_Attribute_Class(data_Test, "income")

In [8]:
#Select model
model = GaussianNB()
model.fit(x_Training, y_Training)

#prediction
y_Predic = model.predict(x_Test)

In [9]:
# วัดประสิทธิภาพของ model เปรียบเทียบกับผลเฉลย y_Test โดยเราจะดู Accuracy Score หรือ classification_report
print("Accuracy -> {0:.2f} %".format(accuracy_score(y_Test, y_Predic)*100))
# print(classification_report(y_Test, y_Predic))
print(pd.crosstab(y_Test, y_Predic, rownames=["Real"], colnames=["Predict"], margins=True))

Accuracy -> 79.60 %
Predict     0     1   All
Real                     
0        7055   349  7404
1        1644   721  2365
All      8699  1070  9769


Accuracy -> 79.38 % ยังถือว่าไม่ผ่านตามกำหนด